In [1]:
# Major usage of: https://gist.github.com/dannguyen/c9cb220093ee4c12b840
guardian_db = '/home/ostapkharysh/Documents/bt_data/DB/Guardian'

In [2]:
import json
import requests
from os import makedirs
from os.path import join, exists, abspath
from datetime import date, timedelta
from newspaper import Article
import pandas as pd
import datetime
from requests.exceptions import HTTPError
from db_manager import *
import re

In [3]:
def get_text(link):
    try:
        r = requests.get(link.strip())
        r.raise_for_status()
        article = Article(link.strip())
        article.download()
        article.parse()
        return article.text
    except HTTPError:
        print("No article found by this link!", article.url) 

In [4]:
#ARTICLES_DIR = join('/home/ostapkharysh/Documents/bt_data/Guardian_news', 'tempdata', 'articles')
#makedirs(ARTICLES_DIR, exist_ok=True)

In [5]:
MY_API_KEY = "0abd707a-96aa-4e21-b5cd-b314c6955753"
API_ENDPOINT = 'http://content.guardianapis.com/search'
my_params = {
    'from-date': "",
    'to-date': "",
    'order-by': "newest",
    'show-fields': 'all',
    'page-size': 200,
    'api-key': MY_API_KEY,
    'lang': 'en'
}

In [6]:
# day iteration from here:
conn = create_connection(guardian_db)

start_date = date(2013, 1, 1)  #month, day
end_date = date(2013,1, 1)
dayrange = range((end_date - start_date).days + 1)
for daycount in dayrange:
    dt = start_date + timedelta(days=daycount)
    datestr = dt.strftime('%Y-%m-%d')
    #fname = join(ARTICLES_DIR, datestr + '.json')
    print("Downloading", datestr)
    all_results = []
    my_params['from-date'] = datestr
    my_params['to-date'] = datestr
    current_page = 1
    total_pages = 1
    while current_page <= total_pages:
        print("...page", current_page)
        my_params['page'] = current_page
        resp = requests.get(API_ENDPOINT, my_params)
        data = resp.json()
        all_results.extend(data['response']['results'])
        # if more than one page
        current_page += 1
        total_pages = data['response']['pages']
        #print(total_pages)

    #with open(fname, 'w') as f:
    #print("Writing to", fname)
    try:
        df = pd.DataFrame(all_results)
        s = [[df.iloc[i]['webPublicationDate'], df.iloc[i]['webTitle'], get_text(df.iloc[i]['webUrl']),
              df.iloc[i]['webUrl']] for i in range(len(df))]
        #print(s[0])
        with conn:
            for el in s:
                news = ( datetime.datetime.strptime(el[0], "%Y-%m-%dT%H:%M:%SZ"), el[1], el[2], el[3])
                create_task(conn, news)
    except HTTPError:
        print("No article found by this link!", article.url)
        #f.write(json.dumps(all_results, indent=2))
close_db(conn)
print("FINISHED!")

...page 1
FINISHED!
